In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
import torch
from src.utils import dataset
from src.optimizers.HypercubeTrainer import HypercubeTrainer

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
def get_model(seed=0, output_dim=10):
    """Returns a simple CNN model."""
    torch.manual_seed(seed)
    model = torch.nn.Sequential(
        torch.nn.Conv2d(1, 8, kernel_size=5, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.Conv2d(8, 1, kernel_size=5, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.Flatten(),
        torch.nn.Linear(576, output_dim),
    ).to(DEVICE)
    return model


In [5]:
model = get_model(output_dim=10)
train_dataset, val_dataset = dataset.get_mnist_dataset()

In [6]:
optimizer = HypercubeTrainer(model, device=DEVICE)
optimizer.set_volume_constrain(1e-4) # start with a small volume at first 
optimizer.train(
    train_dataset, val_dataset, loss_obj=0.1, max_iters=3000, batch_size=64, lr=1e-4
)

100%|██████████| 3000/3000 [02:32<00:00, 19.62it/s, loss=0.473, min_val_acc=0.859]

----------  Training with volume parameter constrain 0.0001 completed with loss  0 ----------
